<a href="https://colab.research.google.com/github/michael-692/ML3/blob/main/Aug_12_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
! pip install datasets transformers
! pip install accelerate -U

# New Section

In [34]:
from huggingface_hub import notebook_login

notebook_login()

In [35]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [36]:
import transformers

print(transformers.__version__)

4.31.0


In [37]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

In [38]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [39]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [40]:
show_random_elements(datasets["train"])

,text
0,"The £ 8 @.@ 5 million Central Link ( Welsh : Ffordd Gyswllt Canolog ) between the Queen 's Gate Roundabout ( 51 @.@ 468399 ° N 3 @.@ 157366 ° W ﻿ / 51 @.@ 468399 ; -3.157366 ﻿ ( Queen 's Gate Roundabout ) ) and the junction on Adam Street ( 51 @.@ 479557 ° N 3 @.@ 167631 ° W ﻿ / 51 @.@ 479557 ; -3.167631 ﻿ ( Adam Street Junction ) ) on the A4160 was opened on 16 February 1989 . When it was first opened it only linked Cardiff city centre with Cardiff Bay . It was not until the Butetown Link was opened on 27 March 1995 that this road started to be used as a link to the motorway network , Penarth and the Vale of Glamorgan . It will not be until the Eastern Bay Link is built , when traffic from the city centre will use it to travel east , that it will be fully used . The length of the A4234 is just 1 @.@ 45 km ( 0 @.@ 90 mi ) and is entirely a two lane dual carriageway with clearway restrictions . \n"
1,Topé con hilo \n
2,= = = Loss = = = \n
3,= = Legacy = = \n
4,"In the 2000 film Hey Ram , the lead character , Saket Ram ( played by Kamal Haasan ) and his friend , Amjad Khan ( played by Shah Rukh Khan ) are shown as employees of Wheeler , who was portrayed by Lewis K. Elbinger , before the 1947 Hindu @-@ Muslim riots . In a 2003 volume of the South Asian Studies journal , Sudeshna Gusha published a research article examining Wheeler 's use of photography in his excavations and publications in the Indian subcontinent . In 2011 , the academic journal Public Archaeology published a research paper by Moshenska and Schadla @-@ Hall that analysed Wheeler 's role in presenting archaeology to the British public . Two years later , the Papers from the Institute of Archaeology issued a short comic strip by Moshenska and Alex Salamunovich depicting Wheeler 's activities in studying the archaeology of Libya during World War II . \n"
5,"Anderson only attended film school for two days , preferring to learn the craft by watching films by the filmmakers he liked , as well as watching films accompanied by director 's audio commentary . Anderson has cited Martin Scorsese , Robert Altman , Jonathan Demme , Stanley Kubrick , Orson Welles and Max Ophüls , as his main influences as a filmmaker . \n"
6,"The five aircraft modified by the factory were given the Klimov RD @-@ 45 engine , the Soviet unlicensed copy of the Nene engine , and all six aircraft , used by the VVS for aircrew familiarization and training . The aircraft completed were later relegated to test duties . One aircraft was used for drone tests and another , redesignated as the Tu @-@ 12LL , mounted various pulse jet engines on a pylon above the fuselage . \n"
7,
8,
9,"In September 2008 , after his appearance on Britain 's Got Talent but before the release of his first album , Johnston embarked on a tour of Norway with the choir , performing at Stavanger Cathedral and Utstein Abbey , among other places . The tour was conceived because the Diocese of Stavanger is connected with the Diocese of Carlisle through the Partnership for World Mission . This was Johnston 's last tour with the choir . Johnston features as head chorister on one of the choir 's albums , The Choral Music of F.W Wadely , released in November 2008 . \n"


In [41]:
model_checkpoint = "distilgpt2"

In [42]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [43]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [44]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [45]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [46]:
# block_size = tokenizer.model_max_length
block_size = 128

In [47]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [48]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [49]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

In [50]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [51]:
from transformers import Trainer, TrainingArguments

In [53]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

ImportError: ignored

In [22]:
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.4 MB/s eta 0:00:00


In [23]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

ImportError: ignored

In [24]:
! pip install accelerate -U

In [25]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)


ImportError: ignored